# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [1]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [14]:
### General parameters
max_steps = 1e7 # Set maximum number of steps to run environment.
run_path = "jump2" # The sub-directory name for model and summary statistics
load_model = True # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 10000 # Frequency at which to save training statistics.
save_freq = 10000 # Frequency at which to save model.
env_name = "build2/jump.exe" # Name of the training environment file.
curriculum_file = "jump2.json"

### Algorithm-specific parameters for tuning
gamma = 0.99 # Reward discount rate.
lambd = 0.95 # Lambda parameter for GAE.
time_horizon = 2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 # Strength of entropy regularization
num_epoch = 5 # Number of gradient descent steps per batch of experiences.
num_layers = 1 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 2048 # How large the experience buffer should be before gradient descent.
learning_rate = 3e-4 # Model learning rate.
hidden_units = 32 # Number of units in hidden layer.
batch_size = 512 # How many experiences per gradient descent update step.
normalize = True

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [15]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'BottleFlipAcademy' started successfully!


Unity Academy name: BottleFlipAcademy
        Number of brains: 1
        Reset Parameters :
		random_direction -> 1.0
		min_scale -> 0.5
		max_distance -> 3.0
Unity brain name: BottleFlipBrain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 2
        Action space type: continuous
        Action space size (per agent): 1
        Memory space size (per agent): 0
        Action descriptions: 


### Train the Agent(s)

In [ ]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Loading Model...
INFO:tensorflow:Restoring parameters from ./models/jump2\model-1140000.cptk


INFO:tensorflow:Restoring parameters from ./models/jump2\model-1140000.cptk


Saved Model
Step: 1150000. Mean Reward: -1.1274067607546439. Std of Reward: 0.7140787458563955.
Saved Model
Step: 1160000. Mean Reward: -1.1737106017191976. Std of Reward: 0.6603585639355769.
Saved Model
Step: 1170000. Mean Reward: -1.110317900378893. Std of Reward: 0.49730658029508906.
Saved Model
Step: 1180000. Mean Reward: -1.1309230411532203. Std of Reward: 0.5367081359857945.
Saved Model
Step: 1190000. Mean Reward: -1.116686114352392. Std of Reward: 0.5397300294990185.
Saved Model
Step: 1200000. Mean Reward: -1.0. Std of Reward: 0.0.
Saved Model
Step: 1210000. Mean Reward: -1.0. Std of Reward: 0.0.
Saved Model
Step: 1220000. Mean Reward: -0.9997777185027118. Std of Reward: 0.025822375622712574.
Saved Model
Step: 1230000. Mean Reward: -1.0141696791181327. Std of Reward: 0.18921837602867425.
Saved Model
Step: 1240000. Mean Reward: -1.0536003600360035. Std of Reward: 0.3477885627681129.
Saved Model
Step: 1250000. Mean Reward: -1.0574351793296595. Std of Reward: 0.3785406625748665.
Sa

INFO:unityagents:
Lesson changed. Now in Lesson 1 : 	random_direction -> 0, min_scale -> 0.9, max_distance -> 1.2


Step: 1340000. Mean Reward: -0.12386107352277853. Std of Reward: 2.5861335020445533.
Saved Model
Step: 1350000. Mean Reward: 0.550770218228498. Std of Reward: 3.8303927874464825.
Saved Model
Step: 1360000. Mean Reward: 1.0501820484407156. Std of Reward: 5.126106289793513.
Saved Model
Step: 1370000. Mean Reward: 2.0794156414762743. Std of Reward: 6.635808178795247.
Saved Model
Step: 1380000. Mean Reward: 3.181113662456946. Std of Reward: 7.303898532768478.
Saved Model


...

Step: 1600000. Mean Reward: 3.5201342281879193. Std of Reward: 6.112883146999933.
Saved Model


INFO:unityagents:
Lesson changed. Now in Lesson 14 : 	random_direction -> 0, min_scale -> 0.7, max_distance -> 3


Step: 1610000. Mean Reward: 0.02213719595517901. Std of Reward: 3.064497838775721.
Saved Model


INFO:unityagents:
Lesson changed. Now in Lesson 15 : 	random_direction -> 0, min_scale -> 0.5, max_distance -> 3


Step: 1620000. Mean Reward: 4.506346967559944. Std of Reward: 6.880833484979833.
Saved Model
Step: 1630000. Mean Reward: 2.3353696741854635. Std of Reward: 5.657123859753659.
Saved Model
Step: 1640000. Mean Reward: 4.297592510031208. Std of Reward: 5.957405915562527.
Saved Model
Step: 1650000. Mean Reward: 1.900997697620875. Std of Reward: 3.9233897309678563.
Saved Model
Step: 1660000. Mean Reward: 4.770833333333333. Std of Reward: 6.576968178170445.
Saved Model
Step: 1670000. Mean Reward: 3.5770758839888757. Std of Reward: 5.7010007303956955.
Saved Model
Step: 1680000. Mean Reward: 0.6416555151978589. Std of Reward: 4.368800099736575.
Saved Model
Step: 1690000. Mean Reward: 1.1642456941274124. Std of Reward: 3.0553023227291525.
Saved Model
Step: 1700000. Mean Reward: 1.3160203139427515. Std of Reward: 4.465132905404935.
Saved Model
Step: 1710000. Mean Reward: 2.0686137506987143. Std of Reward: 5.367643817030653.
Saved Model
Step: 1720000. Mean Reward: 2.6483773837403812. Std of Reward

In [13]:
env.close()

### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [6]:
export_graph(model_path, env_name)

ERROR! Session/line number was not unique in database. History logging moved to new session 32
INFO:tensorflow:Restoring parameters from ./models/jump2\model-1140000.cptk


INFO:tensorflow:Restoring parameters from ./models/jump2\model-1140000.cptk


INFO:tensorflow:Froze 8 variables.


INFO:tensorflow:Froze 8 variables.


Converted 8 variables to const ops.


In [ ]:
print("123")